In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import optuna
import datetime
import joblib
from scipy.stats import pearsonr
from utils import Country_temperature

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
color_pal = sns.color_palette()
pd.options.plotting.backend = "plotly"
plt.style.use('fivethirtyeight')

# HR prediction in France

## Data processing

### HR data

In [6]:
HRE = pd.read_csv('./data_input/HRE_tab.csv')
HRE.index = pd.to_datetime(HRE[['Year','Month','Day']])

### TP data

In [7]:
# Precipitation
TP = pd.read_csv("./data_input/TP_tab.csv")
TP.index = pd.to_datetime(TP[['Year','Month','Day']])
TP.drop(['Year','Month','Day'],axis=1,inplace=True)

In [159]:
aa = TP.rolling(7).sum()

TP.iloc[:6].cumsum()


,CH02,CH03,AL02,AL03,BG33,DE13,CZ05,AT32,CH04,CH05,...,TR42,TR51,TR52,TR10,TR32,PT16,PT11,PT18,PT15,PT17
2014-01-01,1.198886,1.002931,0.118342,0.581515,0.003915,0.764960,0.068982,0.116934,0.871517,0.604359,...,0.304724,0.451596,0.499340,0.150345,0.338869,4.789311,8.565561,0.061441,0.067327,0.062805
2014-01-02,7.394544,5.858352,0.380506,0.769193,0.004317,4.412768,0.520379,0.692781,5.008522,2.891195,...,0.618797,2.531071,5.931308,0.221959,3.612911,7.741954,12.641492,0.205572,0.291688,0.524727
2014-01-03,8.446723,6.454642,0.609630,1.010828,0.072775,5.079430,0.761495,0.983310,5.237231,3.023257,...,0.761357,3.876060,10.737308,0.354194,4.284991,22.305510,26.864438,17.962932,14.256358,13.650075
2014-01-04,15.305587,10.883333,0.772148,1.259271,0.110279,7.412576,0.982722,1.651977,8.882706,8.084310,...,1.182557,5.772108,19.014083,0.668621,4.658023,39.516765,47.929227,21.020628,19.956074,19.891455
2014-01-05,24.579763,19.657253,14.366324,10.521351,0.166245,13.214768,2.437180,7.147103,17.399068,17.290230,...,1.379436,5.986198,19.534271,1.046656,4.772935,44.182551,58.530885,21.397829,20.322471,20.927666
2014-01-06,26.182224,20.262528,27.311636,25.822791,0.242241,14.390326,2.823276,7.275053,17.533152,17.347828,...,1.383531,6.272700,19.845786,1.116901,4.811608,65.315389,87.433895,22.273566,20.605821,24.816536


In [161]:
aa.iloc[:6] = TP.iloc[:6].cumsum()
aa

,CH02,CH03,AL02,AL03,BG33,DE13,CZ05,AT32,CH04,CH05,...,TR42,TR51,TR52,TR10,TR32,PT16,PT11,PT18,PT15,PT17
2014-01-01,1.198886,1.002931,0.118342,0.581515,0.003915,0.764960,0.068982,0.116934,0.871517,0.604359,...,0.304724,0.451596,0.499340,0.150345,0.338869,4.789311,8.565561,0.061441,0.067327,0.062805
2014-01-02,7.394544,5.858352,0.380506,0.769193,0.004317,4.412768,0.520379,0.692781,5.008522,2.891195,...,0.618797,2.531071,5.931308,0.221959,3.612911,7.741954,12.641492,0.205572,0.291688,0.524727
2014-01-03,8.446723,6.454642,0.609630,1.010828,0.072775,5.079430,0.761495,0.983310,5.237231,3.023257,...,0.761357,3.876060,10.737308,0.354194,4.284991,22.305510,26.864438,17.962932,14.256358,13.650075
2014-01-04,15.305587,10.883333,0.772148,1.259271,0.110279,7.412576,0.982722,1.651977,8.882706,8.084310,...,1.182557,5.772108,19.014083,0.668621,4.658023,39.516765,47.929227,21.020628,19.956074,19.891455
2014-01-05,24.579763,19.657253,14.366324,10.521351,0.166245,13.214768,2.437180,7.147103,17.399068,17.290230,...,1.379436,5.986198,19.534271,1.046656,4.772935,44.182551,58.530885,21.397829,20.322471,20.927666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-26,3.463217,1.032578,39.414224,39.640724,0.192076,0.126824,0.008292,6.490472,1.351767,9.328983,...,5.599002,12.667215,13.911803,0.560432,3.931493,10.496936,14.385023,2.748373,8.061462,4.821241
2020-04-27,1.716546,0.534059,34.114336,35.565496,0.090870,0.123627,0.008292,6.301429,0.645784,6.339025,...,5.695603,12.445858,12.216186,0.582198,3.933031,8.817310,13.779321,0.630009,5.409536,1.100223
2020-04-28,22.281068,20.127436,15.984505,21.197435,0.089152,12.506370,0.024175,12.634073,20.482253,20.481607,...,3.020058,6.941830,12.232084,0.125444,3.939918,5.081696,11.343062,0.488122,1.987027,0.918981
2020-04-29,35.214284,39.114700,15.287257,21.867899,0.168316,32.478594,1.616267,28.591289,41.248493,39.968263,...,0.147776,1.038650,7.661783,0.039452,3.079984,5.453206,12.238818,0.640417,1.901113,1.286823


In [215]:
n1 = 30
TP_cum_month = TP.rolling(n1).sum()
TP_cum_month.iloc[:n1-1] = TP.iloc[:n1-1].cumsum()

In [217]:
n2 = 7
TP_cum_3d = TP.rolling(n2).sum()
TP_cum_3d.iloc[:n2-1] = TP.iloc[:n2-1].cumsum()

In [216]:
TP_cum_month[Country_temperature(TP,'FR')].plot()

In [102]:
# Temperature
TA = pd.read_csv("./data_input/TA_tab.csv")
TA.index = pd.to_datetime(TA[['Year','Month','Day']])
TA.drop(['Year','Month','Day'],axis=1,inplace=True)

### Features

In [103]:
from utils import temperature_pca

Features creation

In [307]:
# Time Features
Features = HRE[[]].copy()
# Features['quarter'] = Features.index.quarter
# Features['month'] = Features.index.month
Features['dayofyear'] = Features.index.dayofyear
Features['dayofweek'] = Features.index.dayofweek
Features['weekofyear'] = Features.index.isocalendar().week.astype("int64")
# # Precipitation PCA on all location features
# Features = Features.merge(
#     temperature_pca(TP_cum_3d,0.95,'FR',prefix='TP_cum_3d'),
#     how='left',
#     left_index=True,
#     right_index=True
#     )
# Precipitation PCA on all location features
Features = Features.merge(
    temperature_pca(TA,1,'all',prefix='TA'),
    how='left',
    left_index=True,
    right_index=True
    )
Features = Features.merge(
    temperature_pca(TA,1,'FR',prefix='TA'),
    how='left',
    left_index=True,
    right_index=True
    )
Features = Features.merge(
    TP_cum_month[Country_temperature(TP,'FR')],
    how='left',
    left_index=True,
    right_index=True
    )
print(f"{Features.shape[1]} Features : {Features.columns.to_list()}")

27 Features : ['dayofyear', 'dayofweek', 'weekofyear', 'TA_all_PCA_1', 'TA_FR_PCA_1', 'FRB0', 'FR10', 'FRJ1', 'FRI2', 'FRD2', 'FRI3', 'FRD1', 'FRI1', 'FRC1', 'FRJ2', 'FRC2', 'FRK1', 'FRF3', 'FRL0', 'FRH0', 'FRE2', 'FRF1', 'FRG0', 'FRF2', 'FRK2', 'FRM0', 'FRE1']


In [308]:
X_train = Features.query('index >= 2015&index < 2019')
y_train = HRE[['FR']].query('index >= 2015&index < 2019')

X_test = Features.query('index >= 2019&index < "2020-03-01"')
y_test = HRE[['FR']].query('index >= 2019&index < "2020-03-01"')

In [309]:
study = optuna.create_study(direction='minimize', study_name="HRE-experiment")

[I 2022-12-27 19:55:01,954] A new study created in memory with name: HRE-experiment


In [310]:
def objective(trial):
    params = {
        'learning_rate' : trial.suggest_float('learning_rate', 1e-5, 1e-2),
        'n_estimators' : trial.suggest_int('n_estimators', 500, 2000),
        'max_depth' : trial.suggest_int('max_depth', 2, 15),
        'importance_type' : trial.suggest_categorical('importance_type', ['weight', 'gain', 'cover']),
        'reg_lambda' : trial.suggest_float('lambda', 1e-8, 1.0),
        'reg_alpha' : trial.suggest_float('alpha', 1e-8, 1.0)
    }

    # Create an XGBRegressor model with the sampled hyperparameters
    model = XGBRegressor(**params)
    
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Calculate the mean squared error on the validation set
    mse = mean_squared_error(y_test, model.predict(X_test))
    
    # Return the negative MSE to minimize the objective
    return mse

In [311]:
# Get the current date and time
now = datetime.datetime.now()
# Format the current date and time as a string without the seconds
now_str = now.strftime("%Y-%m-%d %H_%M")
n_trials = 200
study = optuna.create_study(direction='minimize',study_name=now_str+f'--{n_trials}')

[I 2022-12-27 19:55:02,078] A new study created in memory with name: 2022-12-27 19_55--200


In [321]:
study.optimize(objective, n_trials=n_trials)

[I 2022-12-27 20:21:43,332] Trial 200 finished with value: 0.00730586973599402 and parameters: {'learning_rate': 0.007947945411130536, 'n_estimators': 937, 'max_depth': 12, 'importance_type': 'cover', 'lambda': 0.9578096193308043, 'alpha': 0.3173410890835323}. Best is trial 198 with value: 0.0056838193172590295.
[I 2022-12-27 20:21:47,954] Trial 201 finished with value: 0.007364634592331626 and parameters: {'learning_rate': 0.006806243609081707, 'n_estimators': 845, 'max_depth': 12, 'importance_type': 'cover', 'lambda': 0.9793471489270782, 'alpha': 0.14322751798333355}. Best is trial 198 with value: 0.0056838193172590295.
[I 2022-12-27 20:21:52,831] Trial 202 finished with value: 0.006006134413081978 and parameters: {'learning_rate': 0.007871360687545374, 'n_estimators': 990, 'max_depth': 13, 'importance_type': 'cover', 'lambda': 0.9384404876422876, 'alpha': 0.9663781601213104}. Best is trial 198 with value: 0.0056838193172590295.
[I 2022-12-27 20:21:56,817] Trial 203 finished with val

In [322]:
# Get the current date and time
now = datetime.datetime.now()
# Format the current date and time as a string without the seconds
now_str = now.strftime("%Y-%m-%d %H_%M")
joblib.dump(study, f"./OPTUNA_results/{now_str}--{2*n_trials}.pkl")

['./OPTUNA_results/2022-12-27 20_54--400.pkl']

In [323]:
best_params = study.best_params
model = XGBRegressor(**best_params)
model.fit(X_train,y_train)

# Evaluate the model on the test set
test_mse = mean_squared_error(y_test, model.predict(X_test))
print(f'Test MSE: {test_mse:.5f}')
print(f'Test RMSE: {np.sqrt(test_mse):.5f}')
print(f'Test R: {pearsonr(y_test.to_numpy().reshape(-1), model.predict(X_test))[0]:.3f}')
print(f"params = {best_params}")

Test MSE: 0.00546
Test RMSE: 0.07386
Test R: 0.717
params = {'learning_rate': 0.007939871551054469, 'n_estimators': 1237, 'max_depth': 15, 'importance_type': 'gain', 'lambda': 0.8282426966499562, 'alpha': 0.00028955386615664306}


In [324]:
n1 = 30
TP_cum_month = TP.rolling(n1).sum()
TP_cum_month.iloc[:n1-1] = TP.iloc[:n1-1].cumsum()

In [325]:
from utils import features_importance
features_importance(model)

In [326]:
y_predict = model.predict(X_test)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = HRE.index,
        y = HRE.FR,
        name = 'Truth Data'
    )
)
fig.add_trace(
    go.Scatter(
        x = X_test.index,
        y = y_predict,
        name = 'Prediction'
    )
)
fig.update_layout(
    title = f"RMSE for 2019 forecast : {np.round(np.sqrt(mean_squared_error(y_test,y_predict)),5)}",
    hovermode='x'
)

In [327]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(hovermode='x')
fig.update_yaxes(type="log")
fig.show()

In [328]:
optuna.visualization.plot_param_importances(study)

In [329]:
fig = optuna.visualization.plot_slice(study)
fig.update_yaxes(type="log")
fig.show()
